# CoDaS-HEP Columnar Data Analysis, part 1

<br><br><br><br><br>

## Programming paradigms

<br><br><br><br><br>

### Imperative

<br><br><br><br><br>

### Functional

<br><br><br><br><br>

### Object-oriented

<br><br><br><br><br>

### Array-oriented

<br><br><br><br><br>

### Declarative

<br><br><br><br><br>

### ~~Which is best?~~ Which problems are each best suited for?

<br><br><br><br><br>

## Array-oriented programming

<br><br><br><br><br>

### History

<br><br><br><br><br>

### Synergy with data analysis

<br><br><br><br><br>

### Array-oriented in Python: NumPy and everything else

<br><br><br><br><br>

### Limitations of array-oriented programming

<br><br><br><br><br>

### Accelerating code in Python

<br><br><br><br><br>

## From ROOT files into arrays

<br><br><br><br><br>

### Uproot

<br><br><br><br><br>

### Awkward Array

<br><br><br><br><br>

## Project: H → ZZ → 4ℓ

<br><br><br><br><br>

### 4 leptons of the same flavor

<br><br><br><br><br>

### Opposite charges

<br><br><br><br><br>

### On your own: the H → ZZ → 2μ2e case

<br><br><br><br><br>

### Hint!